In [1]:
import numpy as np
import scipy as scp
import scipy.stats as ss
import sklearn.metrics as sm

from scipy.integrate import quad

import matplotlib.pyplot as plt
import scipy.special as scps
from statsmodels.graphics.gofplots import qqplot
from scipy.linalg import cholesky
from functools import partial
from scipy.optimize import minimize
from IPython.display import display
import sympy; sympy.init_printing()

def display_matrix(m):
    display(sympy.Matrix(m))

In [2]:
mu = 0.05                                          # drift
rho = -0.8                                         # correlation coefficient
kappa = 3                                          # mean reversion coefficient
theta = 0.1                                        # long-term mean of the variance
sigma = 0.25                                       # (Vol of Vol) - Volatility of instantaneous variance
T = 15                                             # Time to mat
r = 0.05                                           # risk-free rate
K = 100                                            # Stike  
v0 = 0.08                                          # spot variance
S0 = 100                                           # spot stock price 
k = np.log(K/S0) 


## Heston characteristic function as proposed by Schoutens (2004)

In [2]:
def cf_Heston_schoutens(u, t, v0, mu, kappa, theta, rho, sigma):
    """
    Heston characteristic function as proposed by Schoutens (2004)
    """
    xi = kappa - sigma*rho*u*1j
    d = np.sqrt( xi**2 + sigma**2 * (u**2 + 1j*u) )
    g1 = (xi+d)/(xi-d)
    g2 = 1/g1
    cf = np.exp( 1j*u*mu*t + (kappa*theta)/(sigma**2) * ( (xi-d)*t - 2*np.log( (1-g2*np.exp(-d*t))/(1-g2) ))\
              + (v0/sigma**2)*(xi-d) * (1-np.exp(-d*t))/(1-g2*np.exp(-d*t)) )
    return cf

cf_Heston_schoutens_valuation = partial(cf_Heston_schoutens, t=T, v0=v0, mu=mu, theta=theta, sigma=sigma, kappa=kappa, rho=rho) 

NameError: name 'T' is not defined

In [97]:
def Q1(k, cf, right_lim):
    """
    P(X<k) - Probability to be in the money under the stock numeraire.
    cf: characteristic function
    right_lim: right limit of integration
    """
    integrand = lambda u: np.real( (np.exp(-u*k*1j) / (u*1j)) * 
                                  cf(u-1j) / cf(-1.0000000000001j) )  
    return 1/2 + 1/np.pi * quad(integrand, 1e-15, right_lim, limit=500 )[0]

def Q2(k, cf, right_lim):
    """
    P(X<k) - Probability to be in the money under the money market numeraire
    cf: characteristic function
    right_lim: right limit of integration
    """
    integrand = lambda u: np.real( np.exp(-u*k*1j) /(u*1j) * cf(u) )
    return 1/2 + 1/np.pi * quad(integrand, 1e-15, right_lim, limit=500 )[0]


In [98]:
limit_max = 1000      # right limit in the integration                
call = S0 * Q1(k, cf_Heston_schoutens_valuation, limit_max) - K * np.exp(-r*T)* Q2(k, cf_Heston_schoutens_valuation, limit_max)
print(call)

65.27786862734618


In [99]:
model_price = S0 * Q1(k, cf_Heston_schoutens_valuation, limit_max) - K * np.exp(-r*T) * Q2(k, cf_Heston_schoutens_valuation, limit_max)
print(model_price)

65.27786862734618


In [100]:
def call_price(S0, K, T, v0, r, par): #BUGGY
    limit_max = 1000
    mu, theta, sigma, kappa, rho = par
    cf_Heston_schoutens_valuation = partial(cf_Heston_schoutens, t=T, v0=v0, mu=mu, theta=theta, sigma=sigma, kappa=kappa, rho=rho) 
    model_price = S0 * Q1(k, cf_Heston_schoutens_valuation, limit_max) - K * np.exp(-r*T) * Q2(k, cf_Heston_schoutens_valuation, limit_max)
    return model_price

def call_loss_function(par, **kwargs):
    mu, theta, sigma, kappa, rho = par
    
    S0 = kwargs['S0']
    K =  kwargs['K']
    T =  kwargs['T']
    r =  kwargs['r']
    v0 = kwargs['v0']
    market_price = kwargs['market_price']
    k = np.log(K/S0)
    
    try:
        limit_max = kwargs['limit_max']
    except:
        limit_max = 1000
    
    cf_Heston_schoutens_valuation = partial(cf_Heston_schoutens, t=T, v0=v0, mu=mu, theta=theta, sigma=sigma, kappa=kappa, rho=rho) 
    model_price = S0 * Q1(k, cf_Heston_schoutens_valuation, limit_max) - K * np.exp(-r*T) * Q2(k, cf_Heston_schoutens_valuation, limit_max)
    loss = np.abs((market_price - model_price))
    return loss

# def call_loss_function(S0, K, T, v0, par, market_price):
#     mu, theta, sigma, kappa, rho = par
#     cf_Heston_schoutens_valuation = partial(cf_Heston_schoutens, t=T, v0=v0, mu=r, theta=theta, sigma=sigma, kappa=kappa, rho=rho) 
#     market_price = market_price 
#     model_price = S0 * Q1(k, cf_Heston_schoutens_valuation, limit_max) - K * np.exp(-r*T) * Q2(k, cf_Heston_schoutens_valuation, limit_max)
#     loss = np.abs((market_price - model_price)/market_price)
#     return loss

## Senity Check

In [101]:
# mu, theta, sigma, kappa, rho
mu = 0.05                                          # drift
theta = 0.1                                        # long-term mean of the variance
sigma = 0.25                                       # (Vol of Vol) - Volatility of instantaneous variance
kappa = 3                                          # mean reversion coefficient
rho = -0.8                                         # correlation coefficient
T = 15                                             # Time to mat
r = 0.05                                           # risk-free rate
K = 100                                            # Stike  
v0 = 0.08                                          # spot variance
S0 = 100                                           # spot stock price 
k = np.log(K/S0) 

# Check if the prices match
limit_max = 1000      # right limit in the integration                
call = S0 * Q1(k, cf_Heston_schoutens_valuation, limit_max) - K * np.exp(-r*T)* Q2(k, cf_Heston_schoutens_valuation, limit_max)
print('Call Price from equation 1:',call)

par = (mu, theta, sigma, kappa, rho)
x = call_price(S0, K, T, r, v0, par)
print('Call Price from equation 2:',x)

loss = call_loss_function(par, S0=S0, K=K, T=T, r=r, v0=v0, market_price=call)
print('loss:',loss)

Call Price from equation 1: 65.27786862734618
Call Price from equation 2: 73.99263806326081
loss: 0.0


In [10]:
init_par = np.random.uniform(size=5)
call_loss_function_to_optimize = partial(call_loss_function, S0=S0, K=K, T=T, r=r, v0=v0, market_price=call)
res = minimize(call_loss_function_to_optimize, init_par, method='nelder-mead',
               options={'xtol': 1e-8, 'disp': True})
print(res.x)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in cdouble_scalars
  import sys
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in cdouble_scalars
  import sys
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in cdouble_scalars
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in cdouble_scalars
  if __name__ == '__main__':
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in log
  if __name__ == '__main__':
C:\Users\user\Anaconda3\lib\site-packages\scipy\integrate\quadpack.py:385: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)
C:

Optimization terminated successfully.
         Current function value: 0.000001
         Iterations: 271
         Function evaluations: 530
[ 0.0501164   0.04032133  0.03629767 -0.00800571 -1.00256944]


In [13]:
init_par = np.array([mu, theta, sigma, kappa, rho]) + np.random.normal(0,.1,5)

call_loss_function_to_optimize = partial(call_loss_function, S0=S0, K=K, T=T, r=r, v0=v0, market_price=call)

res = minimize(call_loss_function_to_optimize,
               init_par,

               method='nelder-mead',
               options={'xtol': 1e-8, 'disp': True})
print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 180
         Function evaluations: 354
[ 1.27527685e-01 -2.00234501e-03  2.37961024e-01  2.78034622e+00
 -8.63268715e-01]


In [20]:
# %%time
# init_par = np.array([mu, theta, sigma, kappa, rho]) + np.random.normal(0,.001,5)
# call_loss_function_to_optimize = partial(call_loss_function, S0=S0, K=K, T=T, r=r, v0=v0, market_price=call)
# res = minimize(call_loss_function_to_optimize, init_par, method='nelder-mead',
#                options={'xtol': 1e-15, 'disp': True})
# print('init par:', init_par)
# print('target:', np.array([mu, theta, sigma, kappa, rho]))
# print('result:',res.x)

init par: [ 0.04741062  0.09916593  0.25106994  3.0016109  -0.79876408]
target: [ 0.05  0.1   0.25  3.   -0.8 ]
result: [ 0.04903197  0.09998852  0.24605324  3.07609344 -0.79320324]
Wall time: 32.3 s


In [87]:
%%time
# init_par = np.array([mu, theta, sigma, kappa, rho]) + np.random.normal(0,.01,5)
# init_par = np.random.normal(0,.01,5)

init_par = (np.random.uniform(.05,.95), #mu
            np.random.uniform(.5,5),    #theta
            np.random.uniform(.05,.95), #sigma
            np.random.uniform(.5,5),    #kappa
            np.random.uniform(-.9,-.1)) #rho

call_loss_function_to_optimize = partial(call_loss_function, S0=S0, K=K, T=T, r=r, v0=v0, market_price=call)

res = minimize(call_loss_function_to_optimize, init_par, method='nelder-mead',
              bounds = ((.05,.95),(.5,5),(.05,.95),(.5,5),(-.9,-.1)),
               options={'xtol': 1e-10, 'disp': True})
print('init par:', init_par)
print('target:', np.array([mu, theta, sigma, kappa, rho]))
print('result:',res.x)
print('MSE init par vs target:', sm.mean_squared_error(init_par, np.array([mu, theta, sigma, kappa, rho])))
print('MSE result vs target:', sm.mean_squared_error(res.x, np.array([mu, theta, sigma, kappa, rho])))

C:\Users\user\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:518: RuntimeWarning: Method nelder-mead cannot handle constraints nor bounds.
  RuntimeWarning)


init par: (0.6875867084346896, 3.6448119051680363, 0.6579970507338553, 4.010206603424301, -0.36409940652296613)
target: [ 0.05  0.1   0.25  3.   -0.8 ]
result: [ 0.9502103   2.93505584  0.38628178  2.23087802 -0.92247521]
MSE init par vs target: 2.8698393112393683
MSE result vs target: 1.8946083409178296
Wall time: 26.2 s


In [94]:
call_price(S0, K, T, r, v0, par=res.x)

In [93]:
cf_Heston_schoutens_valuation = partial(cf_Heston_schoutens, t=T, v0=v0, mu=res.x[0], theta=res.x[1], sigma=res.x[2], kappa=res.x[3], rho=res.x[4]) 
call = S0 * Q1(k, cf_Heston_schoutens_valuation, limit_max) - K * np.exp(-r*T)* Q2(k, cf_Heston_schoutens_valuation, limit_max)
print(call)

76.38166980972447


In [88]:
np.array([mu, theta, sigma, kappa, rho])

array([ 0.05,  0.1 ,  0.25,  3.  , -0.8 ])

In [47]:
print(sm.mean_squared_error(init_par, np.array([mu, theta, sigma, kappa, rho])))
print(sm.mean_squared_error(res.x, np.array([mu, theta, sigma, kappa, rho])))

1.3092069719845903e-06
0.0006519744719196647
